# Creamos la estructura de datos para la opción 3

In [4]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras 
import os
import json

import random as python_random
import time
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import MeanAbsolutePercentageError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.models import load_model

from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

from keras import layers, models

np.random.seed(123)
python_random.seed(123)
tf.random.set_seed(1234)

from keras_tuner import HyperModel, HyperParameters
from keras_tuner.tuners import Hyperband

from funcionesComunes import *

2024-11-07 09:56:50.290127: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-07 09:56:50.293372: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-07 09:56:50.303453: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-07 09:56:50.320945: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-07 09:56:50.325927: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-07 09:56:50.344537: I tensorflow/core/platform/cpu_feature_gu

## Entrenamos el modelo

In [13]:
# Función para construir el modelo LSTM basado en la configuración
def build_model_from_config(input_shape, config):
    model = Sequential()
    
    # Añadir capas LSTM iniciales
    for i in range(config["num_lstm_layers"]):
        units = config.get(f"units_lstm_{i}")
        if i == 0:
            model.add(LSTM(units=units, input_shape=input_shape, return_sequences=True, use_bias=True))
        else:
            model.add(LSTM(units=units, return_sequences=True, use_bias=True))
    
    # Capa Dropout
    model.add(Dropout(config["dropout"]))
    
    # Capas LSTM adicionales después de Dropout
    for i in range(config["num_lstm_layers_after"]):
        units = config.get(f"units_lstm_after_{i}")
        return_sequences = (i < config["num_lstm_layers_after"] - 1)  # Última capa no tiene return_sequences
        model.add(LSTM(units=units, return_sequences=return_sequences, use_bias=True))
    
    # Capa Dense
    model.add(Dense(config["dense_units"], activation=config["dense_activation_1"], use_bias=True))
    
    # Segunda capa Dropout
    model.add(Dropout(config["dropout_2"]))
    
    # Capa de salida Dense
    model.add(Dense(1, activation=config["dense_activation_2"], use_bias=True))
    
    # Configurar el optimizador con el learning rate especificado
    optimizer = Adam(learning_rate=config["learning_rate"])
    
    # Compilar el modelo
    model.compile(optimizer=optimizer,
                  loss=MeanSquaredError(),
                  metrics=[MeanSquaredError(name='mse'),
                           RootMeanSquaredError(name='rmse'),
                           MeanAbsolutePercentageError(name='mape')])
    
    return model

In [32]:
WINDOWS_SIZE = 3

# Obtenemos el nombre de archivo
nombreArchivo = "totalMerged_best_models.json".split("_best_models.json")[0]

print(f"PROCESANDO ARCHIVO: {nombreArchivo}")

# Cargamos archivos numpy
data_train_global = np.load(f'RCPMergedTransferTotal/{nombreArchivo}_train_transfer.npz', allow_pickle=True)
data_val_global = np.load(f'RCPMergedTransferTotal/{nombreArchivo}_val_transfer.npz', allow_pickle=True)

X_train_transfer = data_train_global['X_train_transfer']
y_train_transfer = data_train_global['y_train_transfer']
X_val_transfer = data_val_global['X_val_transfer']
y_val_transfer = data_val_global['y_val_transfer']
valorNormalizacion = data_train_global["valorNormalizacion"]

with open("models/LSTMMerged_parallel/totalMerged_best_models.json") as f:
    params = json.load(f)

config = params[0]

model = build_model_from_config((WINDOWS_SIZE + 1, X_train_transfer.shape[2]), config)


model.fit(X_train_transfer, y_train_transfer, epochs=200, batch_size=config["batch_size"],
                validation_data=(X_val_transfer, y_val_transfer), callbacks=[EarlyStopping(monitor='val_loss', patience=20)])

PROCESANDO ARCHIVO: totalMerged
Epoch 1/200


/home/workstation/Escritorio/PabloC/TransferLearning/.venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5355/5355 ━━━━━━━━━━━━━━━━━━━━ 46s 8ms/step - loss: 0.0531 - mape: 88.6525 - mse: 0.0531 - rmse: 0.2294 - val_loss: 0.0469 - val_mape: 87.4840 - val_mse: 0.0469 - val_rmse: 0.2166
Epoch 2/200
5355/5355 ━━━━━━━━━━━━━━━━━━━━ 43s 8ms/step - loss: 0.0477 - mape: 87.7562 - mse: 0.0477 - rmse: 0.2184 - val_loss: 0.0467 - val_mape: 87.8727 - val_mse: 0.0467 - val_rmse: 0.2161
Epoch 3/200
5355/5355 ━━━━━━━━━━━━━━━━━━━━ 43s 8ms/step - loss: 0.0471 - mape: 86.7097 - mse: 0.0471 - rmse: 0.2170 - val_loss: 0.0472 - val_mape: 89.4058 - val_mse: 0.0472 - val_rmse: 0.2172
Epoch 4/200
5355/5355 ━━━━━━━━━━━━━━━━━━━━ 44s 8ms/step - loss: 0.0469 - mape: 86.7213 - mse: 0.0469 - rmse: 0.2166 - val_loss: 0.0469 - val_mape: 88.5503 - val_mse: 0.0469 - val_rmse: 0.2166
Epoch 5/200
5355/5355 ━━━━━━━━━━━━━━━━━━━━ 44s 8ms/step - loss: 0.0466 - mape: 86.1260 - mse: 0.0466 - rmse: 0.2159 - val_loss: 0.0465 - val_mape: 86.0774 - val_mse: 0.0465 - val_rmse: 0.2156
Epoch 6/200
5355/5355 ━━━━━━━━━━━━━━━━━━━━ 44s 8ms/s

In [34]:
model.save("models/LSTMMerged_parallel/totalMerged_best_models.keras")

# Aplicamos transfer

In [35]:
# Creamos un df que contiene los resultados de los modelos freeze de cada especie
df_Freeze = pd.DataFrame(columns = ["File", "Time", "TrainMSE", "TrainRMSE", "TrainR2", "TrainMAPE", 
                                    "ValidationMSE", "ValidationRMSE", "ValidationR2", "ValidationMAPE",
                                    "TestMSE", "TestRMSE", "TestR2", "TestMAPE"])

for archivo in os.listdir("RCPMergedTransfer"):
    if "train" in archivo: 
        
        nombreArchivo = archivo.split("_train")[0]
        print(f"PROCESANDO ARCHIVO: {nombreArchivo}")

        # Cargar datos
        data_transfer_train = np.load(f'RCPMergedTransferTotal/totalMerged_train_transfer.npz', allow_pickle=True)
        data_transfer_val = np.load(f'RCPMergedTransfer/{nombreArchivo}_val.npz', allow_pickle=True)
        data_transfer_test = np.load(f'RCPMergedTransfer/{nombreArchivo}_test.npz', allow_pickle=True)
        
        # Aqui tenemos que usar TRAIN original para aplicar transfer 
        X_train_transfer = data_transfer_train["X_train_transfer"]
        y_train_transfer = data_transfer_train["y_train_transfer"]

        # Aqui tenemos que usar VAL original para aplicar transfer 
        X_val_transfer = data_transfer_val["X_val"]
        y_val_transfer = data_transfer_val["y_val"]
        
        # Este test es para ver la capacidad del modelo
        X_test_transfer = data_transfer_test["X_test"]
        y_test_transfer = data_transfer_test["y_test"]
        
        # Cargamos el conjunto de datos de val original para hacer el transfer
        model = tf.keras.models.load_model('models/LSTMMerged_parallel/totalMerged_best_models.keras')
        modelFreeze = model
        
        # Cargamos el json para obtener el batch_size
        with open("models/LSTMMerged_parallel/totalMerged_best_models.json") as f:
            params = json.load(f)

        config = params[0]

        # Obtener el optimizador del modelo
        optimizer = modelFreeze.optimizer

        # Obtenemos el batch
        batch_size_LSTM = config["batch_size"]

        # Indicamos el numero de layers a entrenar
        NUM_TRAINABLE = 1

        numLSTM_layers = sum(1 for layer in modelFreeze.layers if "lstm" in layer.name)

        numFreezeLayers = numLSTM_layers - NUM_TRAINABLE # Congelamos todas menos la útlima capa

        numberLayersFreezed = 0

        # Congelar las primeras 'numFreezeLayers' capas LSTM
        for i, layer in enumerate(modelFreeze.layers):

            if "lstm" in layer.name and numFreezeLayers>numberLayersFreezed:
                numberLayersFreezed += 1
                layer.trainable = False

            print(layer, layer.trainable)

        # Compilamos el modelo con los nuevos datos
        modelFreeze.compile(optimizer=optimizer, loss=MeanSquaredError(), metrics=[
                                    MeanSquaredError(name='mse'),
                                    RootMeanSquaredError(name='rmse'),
                                    MeanAbsolutePercentageError(name='mape')
                                ])

        # Comienza a medir el tiempo de entrenamiento
        start_time = time.time()

        historyLSTMTransfer = modelFreeze.fit(X_train_transfer, y_train_transfer, epochs=200, batch_size=batch_size_LSTM,
                                validation_data=(X_val_transfer, y_val_transfer), callbacks=[EarlyStopping(monitor='val_loss', patience=10)])
        
        # Finaliza la medición del tiempo de entrenamiento
        end_time = time.time()
        print(f"[MODELO CONGELADO]: {end_time-start_time}")

        # Guardamos el modelo
        modelFreeze.save(f"models/LSTMMerged_parallel/{nombreArchivo}.keras")

        # Realizar predicciones y calcular métricas para el conjunto de entrenamiento
        predictions_train = predictionForIndividuals(X_train_transfer, y_train_transfer, modelFreeze, batch_size_LSTM)
        predictions_train["PredictionsDenormalize"] = predictions_train.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
        predictions_train["ActualDenormalize"] = predictions_train.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

        train_mse = mean_squared_error(predictions_train["ActualDenormalize"],predictions_train["PredictionsDenormalize"])
        train_rmse = np.sqrt(train_mse)
        train_mape = (np.sum(np.abs(predictions_train["PredictionsDenormalize"] - predictions_train["ActualDenormalize"])) / np.sum(np.abs(predictions_train["ActualDenormalize"]))) * 100
        train_r2 = r2_score(predictions_train["ActualDenormalize"], predictions_train["PredictionsDenormalize"])

        # Realizar predicciones y calcular métricas para el conjunto de validación
        predictions_val = predictionForIndividuals(X_val_transfer, y_val_transfer, modelFreeze, batch_size_LSTM)
        predictions_val["PredictionsDenormalize"] = predictions_val.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
        predictions_val["ActualDenormalize"] = predictions_val.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

        val_mse = mean_squared_error(predictions_val["ActualDenormalize"],predictions_val["PredictionsDenormalize"])
        val_rmse = np.sqrt(val_mse)
        val_mape = (np.sum(np.abs(predictions_val["PredictionsDenormalize"] - predictions_val["ActualDenormalize"])) / np.sum(np.abs(predictions_val["ActualDenormalize"]))) * 100
        val_r2 = r2_score(predictions_val["ActualDenormalize"], predictions_val["PredictionsDenormalize"])

        # Realizar predicciones y calcular métricas para el conjunto de prueba
        predictions_test = predictionForIndividuals(X_test_transfer, y_test_transfer, modelFreeze, batch_size_LSTM)
        predictions_test["PredictionsDenormalize"] = predictions_test.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
        predictions_test["ActualDenormalize"] = predictions_test.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

        test_mse = mean_squared_error(predictions_test["ActualDenormalize"],predictions_test["PredictionsDenormalize"])
        test_rmse = np.sqrt(test_mse)
        test_mape = (np.sum(np.abs(predictions_test["PredictionsDenormalize"] - predictions_test["ActualDenormalize"])) / np.sum(np.abs(predictions_test["ActualDenormalize"]))) * 100
        test_r2 = r2_score(predictions_test["ActualDenormalize"], predictions_test["PredictionsDenormalize"])

        print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Train): {train_mse}, {train_rmse}, {train_r2}, {train_mape}")
        print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Val): {val_mse}, {val_rmse}, {val_r2}, {val_mape}")
        print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Test): {test_mse}, {test_rmse}, {test_r2}, {test_mape}")

        # Guardamos los datos calculados
        df_Freeze.loc[len(df_Freeze)] = [nombreArchivo, end_time-start_time,train_mse, train_rmse, train_r2, train_mape, val_mse, val_rmse, val_r2, val_mape, test_mse, test_rmse, test_r2, test_mape]

df_Freeze.to_csv("resultsTransferOption3/resultados_Option3.csv", index=False)

PROCESANDO ARCHIVO: Cedrus deodara
<LSTM name=lstm_15, built=True> False
<LSTM name=lstm_16, built=True> False
<Dropout name=dropout_7, built=True> True
<LSTM name=lstm_17, built=True> False
<LSTM name=lstm_18, built=True> True
<Dense name=dense_6, built=True> True
<Dropout name=dropout_8, built=True> True
<Dense name=dense_7, built=True> True
Epoch 1/200
5355/5355 ━━━━━━━━━━━━━━━━━━━━ 32s 6ms/step - loss: 0.0178 - mape: 34.5342 - mse: 0.0178 - rmse: 0.1335 - val_loss: 0.0173 - val_mape: 34.7953 - val_mse: 0.0173 - val_rmse: 0.1316
Epoch 2/200
5355/5355 ━━━━━━━━━━━━━━━━━━━━ 31s 6ms/step - loss: 0.0175 - mape: 33.2403 - mse: 0.0175 - rmse: 0.1322 - val_loss: 0.0173 - val_mape: 34.6343 - val_mse: 0.0173 - val_rmse: 0.1316
Epoch 3/200
5355/5355 ━━━━━━━━━━━━━━━━━━━━ 31s 6ms/step - loss: 0.0175 - mape: 33.2241 - mse: 0.0175 - rmse: 0.1322 - val_loss: 0.0173 - val_mape: 34.2759 - val_mse: 0.0173 - val_rmse: 0.1316
Epoch 4/200
5355/5355 ━━━━━━━━━━━━━━━━━━━━ 31s 6ms/step - loss: 0.0175 - mape:

/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of Dat

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 0.016914617228706776, 0.13005620795912348, 0.650492219070725, 22.12599045286961
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 0.017143818938504654, 0.130934407007878, 0.6486137029246042, 21.796490214293197
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 0.01678556041950578, 0.1295591001030255, 0.6500120533686025, 21.834734805400245
PROCESANDO ARCHIVO: Juniperus turkestanica Komar.
<LSTM name=lstm_15, built=True> False
<LSTM name=lstm_16, built=True> False
<Dropout name=dropout_7, built=True> True
<LSTM name=lstm_17, built=True> False
<LSTM name=lstm_18, built=True> True
<Dense name=dense_6, built=True> True
<Dropout name=dropout_8, built=True> True
<Dense name=dense_7, built=True> True
Epoch 1/200
5355/5355 ━━━━━━━━━━━━━━━━━━━━ 30s 5ms/step - loss: 0.0178 - mape: 34.6242 - mse: 0.0178 - rmse: 0.1335 - val_loss: 0.0177 - val_mape: 34.1867 - val_mse: 0.0177 - val_rmse: 0.1330
Epoch 2/200
5355/5355 ━━━━━━━━━━━━━━━━━━━━ 28s 5ms/step - loss: 0.0175 - mape: 32

/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of Dat

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 0.016903583451950525, 0.13001378177697365, 0.6507202106816024, 22.07761206892731
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 0.017538552460873325, 0.13243319999484013, 0.6394617684839412, 22.227044276840346
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 0.017133718514219874, 0.13089583077478012, 0.6419916148629556, 22.138152841263214
PROCESANDO ARCHIVO: Abies pindrow
<LSTM name=lstm_15, built=True> False
<LSTM name=lstm_16, built=True> False
<Dropout name=dropout_7, built=True> True
<LSTM name=lstm_17, built=True> False
<LSTM name=lstm_18, built=True> True
<Dense name=dense_6, built=True> True
<Dropout name=dropout_8, built=True> True
<Dense name=dense_7, built=True> True
Epoch 1/200
5355/5355 ━━━━━━━━━━━━━━━━━━━━ 31s 5ms/step - loss: 0.0178 - mape: 34.6932 - mse: 0.0178 - rmse: 0.1334 - val_loss: 0.0179 - val_mape: 36.3421 - val_mse: 0.0179 - val_rmse: 0.1339
Epoch 2/200
5355/5355 ━━━━━━━━━━━━━━━━━━━━ 29s 5ms/step - loss: 0.0175 - mape: 33.3265 - mse

...

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 0.01687563051463046, 0.12990623739694127, 0.651297803952623, 22.04150188127271
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 0.017573155540844224, 0.13256377914364173, 0.6406665403134971, 22.24877281603308
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 0.017268761591148898, 0.13141066011229416, 0.6390128011359555, 22.250760316534826
PROCESANDO ARCHIVO: Pinus wallichiana
<LSTM name=lstm_15, built=True> False
<LSTM name=lstm_16, built=True> False
<Dropout name=dropout_7, built=True> True
<LSTM name=lstm_17, built=True> False
<LSTM name=lstm_18, built=True> True
<Dense name=dense_6, built=True> True
<Dropout name=dropout_8, built=True> True
<Dense name=dense_7, built=True> True
Epoch 1/200
5355/5355 ━━━━━━━━━━━━━━━━━━━━ 31s 5ms/step - loss: 0.0178 - mape: 34.7336 - mse: 0.0178 - rmse: 0.1335 - val_loss: 0.0178 - val_mape: 36.0513 - val_mse: 0.0178 - val_rmse: 0.1335
Epoch 2/200
5355/5355 ━━━━━━━━━━━━━━━━━━━━ 29s 5ms/step - loss: 0.0175 - mape: 33.1640 - ms

/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of Dat

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 0.016930416457707695, 0.13011693378537512, 0.6501657586256651, 22.126174378140096
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 0.01764890188301877, 0.13284916967380253, 0.6403126062757806, 22.350724619255413
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 0.01717936122959644, 0.1310700622934026, 0.6399095138509182, 22.2219353198516
PROCESANDO ARCHIVO: Juniperus excelsa M.-Bieb
<LSTM name=lstm_15, built=True> False
<LSTM name=lstm_16, built=True> False
<Dropout name=dropout_7, built=True> True
<LSTM name=lstm_17, built=True> False
<LSTM name=lstm_18, built=True> True
<Dense name=dense_6, built=True> True
<Dropout name=dropout_8, built=True> True
<Dense name=dense_7, built=True> True
Epoch 1/200
5355/5355 ━━━━━━━━━━━━━━━━━━━━ 30s 5ms/step - loss: 0.0178 - mape: 34.6165 - mse: 0.0178 - rmse: 0.1334 - val_loss: 0.0178 - val_mape: 34.9667 - val_mse: 0.0178 - val_rmse: 0.1333
Epoch 2/200
5355/5355 ━━━━━━━━━━━━━━━━━━━━ 28s 5ms/step - loss: 0.0176 - mape: 33.15

/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of Dat

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 0.016897695305639737, 0.12999113548869298, 0.6508418778126432, 22.115984367853013
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 0.017597400446943974, 0.13265519381819912, 0.6388550455845334, 22.162110143292225
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 0.01716419099332396, 0.13101217879771315, 0.639301031953541, 22.09023833840815
